In [41]:
import fiona
import numpy as np
import os
import rasterio
from os.path import join
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterstats import zonal_stats
from shapely.geometry import box
from rasterstats import zonal_stats

## Define paths

In [27]:
WORKDIR = "/Users/gis/data"
FLOOD_RASTER_LAYER_PATH = join(WORKDIR, "JRC", "floodMapGL_rp10.tif")
WORLDPOP_RASTER = join(WORKDIR, "worldpop","per_ppp_2020_UNadj.tif")
BOUNDARIES_PATH = join(WORKDIR, "boundaries", "per_adm2.shp")

OUTPUT_FOLDER = join(WORKDIR, "flood_analysis_outputs")
CLIPPED_FLOOD_RASTER = join(OUTPUT_FOLDER, "clipped_flood.tif")
INTERSECTED_FLOOD_RASTER = join(OUTPUT_FOLDER, "intersect.tif")

if not os.path.isdir(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)

## Clip world flood raster using country boundaries

In [28]:
with rasterio.open(FLOOD_RASTER_LAYER_PATH) as src:
    with fiona.open(BOUNDARIES_PATH, "r") as shapefile:
        bbox = box(*shapefile.bounds)
    out_image, out_transform = mask(src, [bbox], crop=True)
    _, height, width = out_image.shape

    metadata = src.meta.copy()
    out_meta = {**metadata, "height": height, "width": width, "transform": out_transform}

    # Binarize.
    bin_image = (out_image > 0).astype(int)
        
    with rasterio.open(CLIPPED_FLOOD_RASTER, "w", **out_meta) as dest:
        dest.nodata = 0
        dest.write(bin_image)

## Upsample

In [46]:
with rasterio.open(CLIPPED_FLOOD_RASTER) as src:
    src_transform = src.transform

    # open input to match
    with rasterio.open(WORLDPOP_RASTER) as match:
        dst_crs = match.crs

        # calculate the output transform matrix
        dst_transform, dst_width, dst_height = calculate_default_transform(
            src.crs,     # input CRS
            dst_crs,     # output CRS
            match.width,   # input width
            match.height,  # input height 
            *match.bounds,  # unpacks input outer boundaries (left, bottom, right, top)
        )

    # set properties for output
    dst_kwargs = src.meta.copy()
    dst_kwargs.update({"crs": dst_crs,
                       "transform": dst_transform,
                       "width": dst_width,
                       "height": dst_height,
                       "nodata": 0})
    print("Coregistered to shape:", dst_height,dst_width,'\n Affine',dst_transform)
    # open output
    with rasterio.open(CLIPPED_FLOOD_RASTER, "w", **dst_kwargs) as dst:
        # iterate through bands and write using reproject function
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

Coregistered to shape: 21977 15214 
 Affine | 0.00, 0.00,-81.33|
| 0.00,-0.00,-0.04|
| 0.00, 0.00, 1.00|


## Find intersecting pixels using worldpop raster

In [47]:
with rasterio.open(CLIPPED_FLOOD_RASTER) as fl_src:
    fl_img = fl_src.read(1)
    
    with rasterio.open(WORLDPOP_RASTER) as wp_src:
        wp_img = wp_src.read(1)        
        wp_img[wp_img < 0] = 0
        assert wp_img.shape == fl_img.shape
            
        res = np.where(fl_img == 1, wp_img, 0)        
        
        height, width = res.shape
        metadata = wp_src.meta.copy()
                
        out_meta = {**metadata, "height": height, "width": width, "transform": wp_src.transform}        
        
        with rasterio.open(INTERSECTED_FLOOD_RASTER, "w", **out_meta) as dest:
            dest.nodata = 0
            dest.write(res, indexes=1)

In [38]:
wp_img.shape

(21977, 15214)

## Run zonal statistics

In [48]:
stats = zonal_stats(BOUNDARIES_PATH, INTERSECTED_FLOOD_RASTER, stats=["sum"], geojson_out=True)

In [49]:
res = {k["properties"]["adm2_name"]: k["properties"]["sum"] for k in stats}

In [50]:
res

{'Abancay': 1699.4576416015625,
 'Acobamba': 635.5490112304688,
 'Acomayo': None,
 'Aija': None,
 'Alto Amazonas': 31628.130859375,
 'Ambo': None,
 'Andahuaylas': 320.6507873535156,
 'Angaraes': 214.17922973632812,
 'Anta': 332.9049072265625,
 'Antabamba': None,
 'Antonio Raymondi': 66.67916107177734,
 'Arequipa': 10364.0966796875,
 'Ascope': None,
 'Asuncion': None,
 'Atalaya': 8151.66259765625,
 'Ayabaca': 335.44195556640625,
 'Aymaraes': 263.64215087890625,
 'Azangaro': 9830.97265625,
 'Bagua': 6302.21435546875,
 'Barranca': None,
 'Bellavista': 19386.126953125,
 'Bolivar': 647.239501953125,
 'Bolognesi': None,
 'Bongara': None,
 'Cajabamba': 234.61306762695312,
 'Cajamarca': None,
 'Cajatambo': None,
 'Calca': 13526.509765625,
 'Callao': None,
 'Camana': 26121.01171875,
 'Canas': 2.5701284408569336,
 'Canchis': None,
 'Candarave': None,
 'Cañete': 11572.263671875,
 'Cangallo': 1153.089599609375,
 'Canta': None,
 'Carabaya': 1628.4200439453125,
 'Caraveli': 0.7215911746025085,
 'Car

In [51]:
sum([v for k, v in res.items() if v is not None])

1131135.115685165